**TODO**
* get rid of requests we don't use
* annotate the code to explain what we're doing and why [tutorial!]
* have to explain what broad categories we wanted to analyze and why
    * clean up our doc about what categories we want to use and add it to the github, explain why we are only using certain [un] categories for now (most bang for our buck)

In [35]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
from datetime import datetime
import re

In [197]:
category_links_df = pd.read_csv('category-links.csv')

visited_categories = pd.read_csv('visited_cats.csv')
resolution_urls = pd.read_csv('res_urls.csv')
total_resolutions = 0

category_links_df.head(5)

,un category,number of resolutions,link
0,HUMAN RIGHTS ADVANCEMENT,223,https://digitallibrary.un.org/search?ln=en&cc=...
1,UNRWA--ACTIVITIES,191,https://digitallibrary.un.org/search?ln=en&cc=...
2,DECOLONIZATION,121,https://digitallibrary.un.org/search?ln=en&cc=...
3,HUMAN RIGHTS--REPORTS,99,https://digitallibrary.un.org/search?ln=en&cc=...
4,APARTHEID,87,https://digitallibrary.un.org/search?ln=en&cc=...


In [204]:
def get_res_links_from_page(page_link):
    
    links = []
    
    page = requests.get(page_link)
    if page.status_code != 200:
        raise Exception("Something went wrong loading category: " + category + ", error code: " + page.status_code)

    print("got page")
    html = page.text
    print("html")
    soup = BeautifulSoup(html, 'html.parser')
    print("soup")
    res_ct_str = soup.find("strong", {"class": None}).text
    print(int(res_ct_str.strip()))
    #resolution_cnt = (int(res_ct_str))
    resolution_cnt = int(soup.find("strong", {"class": None}).text)
    print("res ct:" + str(resolution_cnt))

    if resolution_cnt > 50:
        if resolution_cnt%50 == 0:
            page_count = (int) (resolution_cnt/50)
        else: page_count = int((resolution_cnt/50) + 1)
    else: page_count = 1
    
    print("page ct: ", page_count)

    for page in range(page_count):

        for div in soup.find_all("div", {"class": "moreinfo"}):
            res_link_suffix = div.find("a")["href"]
            links.append('https://digitallibrary.un.org' + res_link_suffix)
        
        print("links length: ",len(links))

        time.sleep(5)

        #load the next page if you're not already on the last page (0 indexing)
        if (page+1 != page_count):
            next_page_link_suffix = soup.find("span", {"class": "rec-navigation"}).findAll("a")[-1]["href"]
            next_page_link = 'https://digitallibrary.un.org' + next_page_link_suffix
            print("page #", page)
            print(next_page_link)

            page = requests.get(next_page_link)
            if page.status_code != 200:
                raise Exception("Something went wrong loading next page, error code: " + page.status_code)

            html = page.text
            soup = BeautifulSoup(html, 'html.parser')
            
        
    if (resolution_cnt != len(links)):
        raise Exception("resolution count does not match")
        
    return links

In [205]:
#Make resolution_urls unique
def unique(list1):
    # initialize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

In [206]:
#get the initial website with the categories
for cat, link in zip (category_links_df["un category"], category_links_df["link"]):
    print(link)
    if (visited_categories[~visited_categories.['visited categories'].isin([cat])]:
        #try:
        print("cat: " + cat)
        cat_res_links = get_res_links_from_page(link) #grab the links, may throw an exception
        print("cat res links len: ", len(cat_res_links))
        resolution_urls = resolution_urls + cat_res_links #append the new links list to the bigger old links list
        visited_categories.append(cat)
        print("success")
        #except :
        #print("oh no the UN blocked you maybe :(")
        #finally:
        time.sleep(5) #delay to hopefully prevent the un from detecting and blocking us

unq_res_urls = unique(resolution_urls)
res_urls_df = pd.DataFrame(unq_res_urls, columns = ['res_urls'])
res_urls_df.to_csv('res_urls.csv', index=False)
visited_cats = pd.DataFrame(visited_categories, columns = ['visited categories'])
visited_cats.to_csv('visited_cats.csv', index=False)

https://digitallibrary.un.org/search?ln=en&cc=Voting%20Data&p=&f=&rm=&ln=en&sf=&so=d&rg=50&c=Voting%20Data&c=&of=hb&fti=0&fct__2=General%20Assembly&fct__9=Vote&fti=0&fct__2=General%20Assembly&fct__9=Vote&fct__8=HUMAN%20RIGHTS%20ADVANCEMENT
https://digitallibrary.un.org/search?ln=en&cc=Voting%20Data&p=&f=&rm=&ln=en&sf=&so=d&rg=50&c=Voting%20Data&c=&of=hb&fti=0&fct__2=General%20Assembly&fct__9=Vote&fti=0&fct__2=General%20Assembly&fct__9=Vote&fct__8=UNRWA--ACTIVITIES
cat: UNRWA--ACTIVITIES
got page
html
soup
191
res ct:191
page ct:  4
links length:  50
page # 0
https://digitallibrary.un.org/search?ln=en&c=Voting+Data&jrec=51&fct__2=General+Assembly&fct__2=General+Assembly&cc=Voting+Data&fct__9=Vote&fct__9=Vote&fct__8=UNRWA--ACTIVITIES
links length:  100
page # 1
https://digitallibrary.un.org/search?ln=en&c=Voting+Data&jrec=101&fct__2=General+Assembly&fct__2=General+Assembly&cc=Voting+Data&fct__9=Vote&fct__9=Vote&fct__8=UNRWA--ACTIVITIES
links length:  150
page # 2
https://digitallibrary.u

In [ ]:
# for all voting data df, index column is the country and each following column is the resolution id. 
# cell is each country's vote (Y if yes, N if no, A if abstain, NP if not present, or NaN if the country didn't exist to vote at the time
all_voting_data = pd.DataFrame(columns = ['country'])

## rows are resolution id's, columns are names of resolutions and years they were voted on
all_res_data = pd.DataFrame(columns = ['Resolution ID', 'Resolution Name', 'Year'])

In [50]:
#Get vote data from each resolution

#resolution df (x= resolution index, y = name, year)
#country_votes df (row = country, col= resolution index) row,col = vote

for res_url in unq_res_urls:
    res_page = requests.get(res_url)
    if res_page.status_code != 200:
        raise Exception("Something went wrong loading resolution, error code: " + res_page.status_code)
        
    html = res_page.text
    soup = BeautifulSoup(html, 'html.parser')

    metadata = soup.find_all("span", {"class": "value col-xs-12 col-sm-9 col-md-10"})

    #number of metadata sections changes from resolution to resolution, so start counting from bottom for some
    res_title = metadata[0] 
    #res_title.text
    vote_summary = metadata[len(metadata) -4]
    vote_date = metadata[len(metadata) -3]
    vote = metadata[len(metadata) -2]
    time.sleep(10) #can change later
    
    #STORE THIS INFO IN DF EACH TIME

KeyboardInterrupt: 

In [158]:
res_page = requests.get('https://digitallibrary.un.org/record/671966?ln=en')
if res_page.status_code != 200:
    raise Exception("Something went wrong loading resolution, error code: " + res_page.status_code)

html = res_page.text
soup = BeautifulSoup(html, 'html.parser')

metadata = soup.find("div", {"id" : "details-collapse"})
#metadata

#checking to make sure the vote was recorded
# we only care about recorded votes since they allow us to track how countries change their views over time
row_content_meta = metadata.find_all("span", {"class" : "value col-xs-12 col-sm-9 col-md-10"})
recorded_vote = [False if 'NON-RECORDED' in row.get_text() else True for row in row_content_meta]

rows = metadata.find_all("div", {"class" : "metadata-row"})

title = ""
res_id = ""
date = ""
vote_table = ""

if False not in recorded_vote:

    for row in rows:
        row_title = row.find("span", {"class" : "title col-xs-12 col-sm-3 col-md-2"}).text
        row_value = row.find("span", {"class" : "value col-xs-12 col-sm-9 col-md-10"})
        
        #strip newline chars from the string
        row_title = row_title.strip()
        
        if row_title == 'Title':
            title = row_value.text
        elif row_title == 'Resolution':
            res_id = row_value.text
        elif row_title == 'Vote date':
            date = row_value.text
            dt = datetime.strptime(date, "%Y-%m-%d")
        elif row_title == 'Vote':
            vote_table = row_value
            # save the html to add country individual vote data to voting db
            # iterate through br's to do this more easily
            

#this_res_data = pd.DataFrame({'Resolution ID': [res_id],#
#                    'Resolution Name' : [title],
#                    'Year' : [dt.year]})

#all_res_data = pd.concat([all_res_data, this_res_data], ignore_index = True, axis = 0)
#all_res_data
            


# perform an outer merge
#all_voting_data.merge(res_voting_data, how='outer', on='country')
 


#all_res_data.drop_duplicates(inplace = True)
# need to drop duplicate columns (not column names but columns where name and all votes are duplicated to be sure)
# all_voting_data.drop_duplicates(

#all_res_data.to_csv('all_res_data.csv', index=False)
#all_voting_data.to_csv('all_voting_data.csv', index=False)

In [162]:
#get vote information into a dataframe for each 
vts = str(vote_table)

for i in range(len(vts)):
    if vts[i] in ['Y', 'N', 'A']:
        vts = vts[:i]+'<br/> ' + vts[i:]
        break
    elif vts[i:i+2] == '> ':
        vts = vts[:i]+'<br/> ' + vts[i:]
        break

vote_table = BeautifulSoup(vts, 'html.parser')

#assume this is populated for now
res_voting_data = pd.DataFrame(columns = ['country', res_id])

for br in vote_table.findAll('br'):
    next_s = br.nextSibling
    if next_s[0:1] in ['Y', 'N', 'A']:
        vote = next_s[0:1].strip()
        print("vote: " + vote)
        country = next_s[2:].strip()
        print("country: " + country)    
    else:
        vote = 'NP'
        country = next_s.strip()
        print("undone country: " + country)
    
    res_voting_data.loc[len(res_voting_data.index)] = [country, vote] 

res_voting_data

undone country: > > > Y AFGHANISTAN
undone country: N ALBANIA
undone country: A ALGERIA
undone country: A ARGENTINA
undone country: Y AUSTRALIA
undone country: Y AUSTRIA
undone country: Y BARBADOS
undone country: Y BELGIUM
undone country: Y BOLIVIA
undone country: Y BOTSWANA
undone country: A BRAZIL
undone country: Y BULGARIA
undone country: A BURMA
undone country: A BURUNDI
undone country: Y BYELORUSSIAN SSR
undone country: CAMBODIA
undone country: Y CAMEROON
undone country: Y CANADA
undone country: A CENTRAL AFRICAN REPUBLIC
undone country: Y CEYLON
undone country: Y CHAD
undone country: Y CHILE
undone country: Y CHINA
undone country: Y COLOMBIA
undone country: A CONGO (BRAZZAVILLE)
undone country: Y CONGO (LEOPOLDVILLE)
undone country: Y COSTA RICA
undone country: N CUBA
undone country: Y CYPRUS
undone country: Y CZECHOSLOVAKIA
undone country: Y DAHOMEY
undone country: Y DENMARK
undone country: DOMINICAN REPUBLIC
undone country: Y ECUADOR
undone country: Y EL SALVADOR
undone country

,country,A/RES/2373(XXII)
0,> > > Y AFGHANISTAN,NP
1,N ALBANIA,NP
2,A ALGERIA,NP
3,A ARGENTINA,NP
4,Y AUSTRALIA,NP
...,...,...
119,Y URUGUAY,NP
120,Y VENEZUELA,NP
121,Y YEMEN,NP
122,Y YUGOSLAVIA,NP
